# TVM 入门指南

## 前端导入模型

In [1]:
from env import tvm
import onnx
from utils.loader import img_data

model_path = '../../models/resnet50-v2-7.onnx'
onnx_model = onnx.load(model_path)

from tvm import relay
from tvm.contrib import graph_executor

target = "llvm"
input_name = "data"
shape_dict = {input_name: img_data.shape}

mod, params = relay.frontend.from_onnx(onnx_model, shape_dict)

with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target=target, params=params)

dev = tvm.device(str(target), 0)
module = graph_executor.GraphModule(lib["default"](dev))

dtype = "float32"
module.set_input(input_name, img_data)
module.run()
output_shape = (1, 1000)
tvm_output = module.get_output(0, tvm.nd.empty(output_shape)).numpy()

<module 'tvm' from '/media/pc/data/4tb/lxw/study/tvm/python/tvm/__init__.py'>
<module 'vta' from '/media/pc/data/4tb/lxw/study/tvm/vta/python/vta/__init__.py'>


One or more operators have not been tuned. Please tune your model for better performance. Use DEBUG logging level to see more details.


In [3]:
import tvm.auto_scheduler as auto_scheduler
from tvm.autotvm.tuner import XGBTuner
from tvm import autotvm

number = 10
repeat = 1
min_repeat_ms = 0  # since we're tuning on a CPU, can be set to 0
timeout = 10  # in seconds

# create a TVM runner
runner = autotvm.LocalRunner(
    number=number,
    repeat=repeat,
    timeout=timeout,
    min_repeat_ms=min_repeat_ms,
    enable_cpu_cache_flush=False,
)

tuning_option = {
    "tuner": "xgb",
    "trials": 10,
    "early_stopping": 100,
    "measure_option": autotvm.measure_option(
        builder=autotvm.LocalBuilder(build_func="default"), runner=runner
    ),
    "tuning_records": "resnet-50-v2-autotuning.json",
}

# 首先从 onnx 模型中提取任务
tasks = autotvm.task.extract_from_program(mod["main"], target=target, params=params)

# 按顺序调优提取的任务
for i, task in enumerate(tasks):
    # prefix = "[Task %2d/%2d] " % (i + 1, len(tasks))
    # tuner_obj = XGBTuner(task, loss_type="rank")
    # tuner_obj.tune(
    #     n_trial=min(tuning_option["trials"], len(task.config_space)),
    #     early_stopping=tuning_option["early_stopping"],
    #     measure_option=tuning_option["measure_option"],
    #     callbacks=[
    #         autotvm.callback.progress_bar(tuning_option["trials"], prefix=prefix),
    #         autotvm.callback.log_to_file(tuning_option["tuning_records"]),
    #     ]
    # )
    break

In [4]:
from tvm.rpc.tracker import Tracker

In [5]:
tracker = Tracker(port=9000, port_end=10000, silent=True)

/home/pc/vscodex/tvm38/bin/python: Error while finding module specification for 'tvm.exec.popen_worker' (ModuleNotFoundError: No module named 'tvm')


ChildProcessError: Subprocess terminated